In [1]:
import pandas as pd
import numpy as np
import re
import unicodedata # Para remoção robusta de acentos

# --- 1. Carregamento dos Dados ---
file_path_excel = 'Vendas Fibratur Teste (1) (2).xlsx' # <-- SEU ARQUIVO EXCEL

try:
    df = pd.read_excel(file_path_excel)
    print(f"Arquivo Excel '{file_path_excel}' carregado com sucesso.")
except FileNotFoundError:
    print(f"Erro: Arquivo Excel '{file_path_excel}' não encontrado.")
    exit()
except Exception as e:
    print(f"Erro ao carregar o arquivo Excel: {e}")
    exit()

print("\n--- Colunas Originais ---")
print(df.columns)

# --- 2. Limpeza e Transformação dos Dados ---
df_cleaned = df.copy()

# 2.1 Padronizar nomes das colunas (Função de Limpeza Geral)
def sanitize_column_names(col_name):
    col_name = str(col_name).lower() # Garante string e minúsculas
    col_name = re.sub(r'\s+', '_', col_name) # Espaços -> underscore
    col_name = re.sub(r'[()]', '', col_name) # Remove parênteses
    col_name = re.sub(r'[?/$]', '', col_name) # Remove caracteres especiais
    # Remove acentos
    col_name = unicodedata.normalize('NFKD', col_name).encode('ascii', 'ignore').decode('ascii')
    # Remove '_r_' ou '_rs' que possam ter vindo de (R$)
    col_name = col_name.replace('_r_', '').replace('_rs', '')
    return col_name

# Aplica a sanitização geral
df_cleaned.columns = [sanitize_column_names(col) for col in df_cleaned.columns]

# **Ajuste Adicional e Específico para garantir os nomes corretos**
# Renomeia explicitamente se algum nome ainda estiver ligeiramente diferente
# (Ex: se 'valor_da_venda_r' foi gerado em vez de 'valor_da_venda_rs')
rename_map = {}
for col in df_cleaned.columns:
    if col.startswith('valor_da_venda'):
        rename_map[col] = 'valor_da_venda'
    elif col.startswith('valor_recebido'):
        rename_map[col] = 'valor_recebido'

if rename_map:
    df_cleaned.rename(columns=rename_map, inplace=True)
    print("\n--- Nomes das colunas monetárias ajustados ---")

print("\n--- Nomes das Colunas Finais Padronizados ---")
print(df_cleaned.columns)

# Verificar se as colunas esperadas existem após renomear
expected_cols = ['codigo_da_venda', 'data_da_venda', 'emissor', 'cliente', 'destino', 'fornecedor', 'valor_da_venda', 'valor_recebido', 'venda_cancelada', 'venda_baixada']
current_cols = df_cleaned.columns.tolist()
missing_cols = [col for col in expected_cols if col not in current_cols]
if missing_cols:
    print(f"\nAviso: Colunas esperadas não encontradas: {missing_cols}.")
extra_cols = [col for col in current_cols if col not in expected_cols]
if extra_cols:
    print(f"\nAviso: Colunas extras encontradas: {extra_cols}.")


# 2.2 Converter 'data_da_venda' para datetime e depois formatar como string DD/MM/YYYY
if 'data_da_venda' in df_cleaned.columns:
    print("\nConvertendo e formatando datas...")
    # Converte para datetime (robusto a formatos comuns)
    df_cleaned['data_da_venda_dt'] = pd.to_datetime(df_cleaned['data_da_venda'], errors='coerce')
    failed_conversions = df_cleaned['data_da_venda_dt'].isnull().sum()
    if failed_conversions > 0:
         print(f"Aviso: Falha ao converter {failed_conversions} datas. Verifique os formatos originais.")

    # Formata para string DD/MM/YYYY (ignora erros/NaT)
    df_cleaned['data_da_venda'] = df_cleaned['data_da_venda_dt'].dt.strftime('%d/%m/%Y')
    # Preenche NaNs que podem ter sido gerados por strftime em NaT com uma string vazia ou outro placeholder
    df_cleaned['data_da_venda'].fillna('Data Inválida', inplace=True)
    df_cleaned.drop(columns=['data_da_venda_dt'], inplace=True) # Remove coluna datetime auxiliar
    print("Coluna 'data_da_venda' formatada como DD/MM/YYYY (tipo string).")
else:
    print("\nAviso: Coluna 'data_da_venda' não encontrada.")


# 2.3 Limpar, Converter e Formatar Colunas Numéricas ('valor_da_venda', 'valor_recebido')
def clean_numeric_value(value):
    if pd.isna(value): return np.nan
    if isinstance(value, (int, float)): return float(value)

    s_value = str(value).strip()
    s_value = re.sub(r'R\$', '', s_value).strip()

    # Lógica aprimorada para pontos e vírgulas
    has_comma = ',' in s_value
    has_dot = '.' in s_value

    if has_comma and has_dot: # Ex: 1.234,56 ou 1,234.56
        comma_pos = s_value.rfind(',')
        dot_pos = s_value.rfind('.')
        if comma_pos > dot_pos: # Vírgula é decimal: 1.234,56
            s_value = s_value.replace('.', '') # Remove milhar
            s_value = s_value.replace(',', '.') # Troca decimal
        else: # Ponto é decimal: 1,234.56 (menos comum no Brasil, mas possível)
            s_value = s_value.replace(',', '') # Remove milhar
            # Ponto já está como decimal
    elif has_comma: # Só vírgula: 1234,56
        s_value = s_value.replace(',', '.') # Troca decimal
    elif has_dot: # Só ponto: 1234.56 ou 2.000
        # Se o último ponto for seguido por exatamente 2 dígitos, assume decimal
        last_dot_pos = s_value.rfind('.')
        if len(s_value) - last_dot_pos - 1 == 2:
             pass # Ponto já está como decimal (ex: 1200.50)
        else:
             # Assume ponto como milhar (ex: 2.000 ou 1234.5) -> remove
             s_value = s_value.replace('.', '')

    try:
        return float(s_value)
    except ValueError:
        return np.nan

numeric_cols = ['valor_da_venda', 'valor_recebido']
for col in numeric_cols:
    if col in df_cleaned.columns:
        print(f"Limpando e formatando coluna monetária: {col}")
        original_nan_count = df_cleaned[col].isnull().sum()
        # Aplica a função de limpeza
        df_cleaned[col] = df_cleaned[col].apply(clean_numeric_value)
        # Garante que a coluna seja numérica, forçando erros para NaN
        df_cleaned[col] = pd.to_numeric(df_cleaned[col], errors='coerce')
        # **Arredonda para 2 casas decimais**
        df_cleaned[col] = df_cleaned[col].round(2)
        new_nan_count = df_cleaned[col].isnull().sum()
        # Verifica se o tipo da coluna é float
        if not pd.api.types.is_float_dtype(df_cleaned[col]):
             print(f"Aviso: A coluna '{col}' não é do tipo float após a limpeza.")
        if new_nan_count > original_nan_count:
            print(f"Aviso: {new_nan_count - original_nan_count} novos NaNs em '{col}' após limpeza/conversão.")
    else:
        print(f"Aviso: Coluna monetária esperada '{col}' não encontrada.")


# 2.4 Padronizar Colunas de Texto (Emissor, Cliente, Destino, Fornecedor)
# (Mantendo a mesma lógica da versão anterior)
text_cols = ['emissor', 'cliente', 'destino', 'fornecedor']
for col in text_cols:
    if col in df_cleaned.columns:
        print(f"Padronizando coluna de texto: {col}")
        df_cleaned[col] = df_cleaned[col].astype(str)
        df_cleaned[col].replace(['nan', 'None', np.nan], 'Desconhecido', inplace=True)
        df_cleaned[col] = df_cleaned[col].str.strip().str.title()
        # (Opcional) Imprimir valores únicos para verificação
        # if df_cleaned[col].nunique() < 50: print(f"--- Valores Únicos em '{col}': {df_cleaned[col].unique()}")
    else:
        print(f"Aviso: Coluna de texto esperada '{col}' não encontrada.")

# 2.5 Padronizar Colunas Booleanas (Venda Cancelada, Venda Baixada)
# (Mantendo a mesma lógica da versão anterior)
boolean_cols = ['venda_cancelada', 'venda_baixada']
mapping = {
    'sim': True, 's': True, 'true': True, '1': True, 'verdadeiro': True,
    'nao': False, 'não': False, 'n': False, 'false': False, '0': False, 'falso': False,
    'nan': np.nan, 'none': np.nan, '': np.nan
}
for col in boolean_cols:
     if col in df_cleaned.columns:
        print(f"Padronizando coluna booleana: {col}")
        original_nan_count = df_cleaned[col].isnull().sum()
        df_cleaned[col] = df_cleaned[col].astype(str).str.lower().str.strip().map(mapping)
        new_nan_count = df_cleaned[col].isnull().sum()
        # (Opcional) Aviso sobre valores não mapeados
        # unmapped_count = new_nan_count - df_cleaned[col].isna().sum()
        # if unmapped_count > 0 : print(f"Aviso: {unmapped_count} valores não reconhecidos em '{col}'.")
     else:
        print(f"Aviso: Coluna booleana esperada '{col}' não encontrada.")


# --- 3. Verificação Final Pós-Limpeza ---
print("\n--- Informações Finais do DataFrame Limpo ---")
df_cleaned.info() # Verificar Dtypes: float64 para valores, object para data, bool para booleanas (se não houver NaN)

print("\n--- Contagem Final de Valores Ausentes ---")
print(df_cleaned.isnull().sum()) # Verificar NaNs

print("\n--- Primeiras 5 linhas do DataFrame Final Limpo e Formatado ---")
print(df_cleaned.head())

# --- 4. Salvar o DataFrame Limpo ---
try:
    output_filename = 'dados_fibratur_limpos_final.xlsx'
    df_cleaned.to_excel(output_filename, index=False)
    print(f"\nDataFrame limpo salvo como '{output_filename}'")
except Exception as e:
    print(f"\nErro ao salvar o DataFrame limpo: {e}")

# --- Fim do Script ---
print("\nProcesso de limpeza e formatação concluído.")

Arquivo Excel 'Vendas Fibratur Teste (1) (2).xlsx' carregado com sucesso.

--- Colunas Originais ---
Index(['Código da Venda', 'Data da Venda', 'Emissor', 'Cliente', 'Destino',
       'Fornecedor', 'Valor da Venda (R$)', 'Valor Recebido (R$)',
       'Venda Cancelada', 'Venda Baixada'],
      dtype='object')

--- Nomes das colunas monetárias ajustados ---

--- Nomes das Colunas Finais Padronizados ---
Index(['codigo_da_venda', 'data_da_venda', 'emissor', 'cliente', 'destino',
       'fornecedor', 'valor_da_venda', 'valor_recebido', 'venda_cancelada',
       'venda_baixada'],
      dtype='object')

Convertendo e formatando datas...
Coluna 'data_da_venda' formatada como DD/MM/YYYY (tipo string).
Limpando e formatando coluna monetária: valor_da_venda
Limpando e formatando coluna monetária: valor_recebido
Padronizando coluna de texto: emissor
Padronizando coluna de texto: cliente
Padronizando coluna de texto: destino
Padronizando coluna de texto: fornecedor
Padronizando coluna booleana: ve

C:\Users\cairo\AppData\Local\Temp\ipykernel_8296\4259248748.py:80: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cleaned['data_da_venda'].fillna('Data Inválida', inplace=True)
C:\Users\cairo\AppData\Local\Temp\ipykernel_8296\4259248748.py:152: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves a

In [10]:
display(df_cleaned)

,codigo_da_venda,data_da_venda,emissor,cliente,destino,fornecedor,valor_da_venda,valor_recebido,venda_cancelada,venda_baixada
0,VD00111,11/02/2025,Ana,Órgão Público,Foz Do Iguaçu,Trend,3879.14,2936.92,False,True
1,VD00420,22/02/2025,João,Órgão Público,Fortaleza,Cvc,5176.17,3675.19,False,True
2,VD00566,18/02/2025,Bruna,Órgão Público,Florianópolis,Trend,3183.51,2687.62,False,False
3,VD00078,03/01/2025,Bruna,Empresa Privada,Florianópolis,Decolar,3269.27,2529.91,False,True
4,VD00182,08/11/2024,João,Pessoa Física,Foz Do Iguaçu,Cvc,3997.31,3667.54,False,True
...,...,...,...,...,...,...,...,...,...,...
595,VD00072,25/12/2024,Fernanda,Pessoa Física,Salvador,Visual Turismo,4181.74,4017.51,False,True
596,VD00107,07/02/2025,João,Órgão Público,Fortaleza,Visual Turismo,4236.70,3753.24,False,True
597,VD00271,23/02/2025,João,Órgão Público,Foz Do Iguaçu,Orinter,4382.18,3038.46,False,True
598,VD00436,12/03/2025,João,Órgão Público,Florianópolis,Orinter,4116.02,4043.02,False,True
